In [208]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [209]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [210]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [211]:
df=pd.read_csv("../input/netflix-original-films-imdb-scores/NetflixOriginals.csv", encoding = "ISO-8859-1")

In [212]:
df.head()

In [213]:
df.columns = [i.replace(' ', '_') for i in df.columns]

In [214]:
# keşifçi veri analizi
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [215]:
check_df(df)

Veri setine göre uzun soluklu filmler hangi dilde oluşturulmuştur? Görselleştirme yapınız.

In [216]:
uzunsoluklu = df.groupby("Language").agg({"Runtime": "mean"}).sort_values(by="Runtime", ascending=False)

In [217]:
uzunsoluklu.reset_index(inplace=True)

In [218]:
print(uzunsoluklu)

In [219]:

f=plt.figure(figsize=(12,12))

plt.barh(uzunsoluklu.Language,uzunsoluklu.Runtime,
        color="#f05131",label="Runtime")

plt.ylabel("Language")
plt.xlabel("Runtime")
plt.legend()
plt.title("uzun soluklu filmler hangi dilde:")
plt.show()

2019 Ocak ile 2020 Haziran tarihleri arasında 'Documentary' türünde çekilmiş filmlerin IMDB değerlerini bulup görselleştiriniz.

In [220]:
import datetime as dt

In [221]:
# premiere tarih değişkenidir türünü düzeltiyorum:
for col in df.columns:
    if "Premiere" in col:
        df[col] = df[col].apply(pd.to_datetime)

In [222]:
documentary_19_20 = df.loc[(df["Premiere"] > "2019-01-31") & (df["Premiere"] < "2020-06-01") & (df["Genre"] == "Documentary")].sort_values(by="Premiere", ascending=False)

In [223]:
print(documentary_19_20)

In [224]:
f=plt.figure(figsize=(12,12))

plt.barh(documentary_19_20.Title,documentary_19_20.IMDB_Score,
        color="#f05131",label="IMDB_Score")

plt.ylabel("Title")
plt.xlabel("IMDB_Score")
plt.legend()
plt.title("2019 Ocak ile 2020 Haziran tarihleri arasında 'Documentary' türünde çekilmiş filmlerin IMDB değerleri")
plt.show()

İngilizce çekilen filmler içerisinde hangi tür en yüksek IMDB puanına sahiptir?

In [225]:
def english_imdb(dataframe):
    dataframe = dataframe.loc[(dataframe["Language"] == "English")].sort_values(by="IMDB_Score", ascending=False)
    return(dataframe["Genre"][0:1])
english_imdb(df)

'Hindi' Dilinde çekilmiş olan filmlerin ortalama 'runtime' suresi nedir?

In [226]:
df.loc[(df["Language"] == "Hindi")].agg({"Runtime": "mean"})

'Genre' Sütunu kaç kategoriye sahiptir ve bu kategoriler nelerdir? Görselleştirerek ifade ediniz.

In [227]:
# kaç kategori:
df["Genre"].nunique() 

In [228]:
# bu kategoriler:
print(df["Genre"].value_counts())

Veri setinde bulunan filmlerde en çok kullanılan 3 dili bulunuz.

In [229]:
df["Language"].value_counts(sort=True)[0:3]

IMDB puanı en yüksek olan ilk 10 film hangileridir?

In [230]:
df.sort_values(by="IMDB_Score", ascending=False).head(10)

IMDB puanı ile 'Runtime' arasında nasıl bir korelasyon vardır? İnceleyip görselleştiriniz.

In [231]:
def correlation_matrix(df, cols):
    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    fig = sns.heatmap(df[cols].corr(), annot=True, linewidths=0.5, annot_kws={'size': 12}, linecolor='w', cmap='RdBu')
    plt.show(block=True)

In [232]:
# kategorik, numerik, numerik görünümlü kategorik ve kategorik ama kardinal değişkenleri düzenleme
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O" and dataframe[col].dtypes != "datetime64[ns]"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    return cat_cols, num_cols, cat_but_car



In [233]:
cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=10, car_th=20)

In [234]:
num_cols

In [235]:
# num_colsta bulunan IMDB puanı ile 'Runtime' arasındaki korelasyon matrisi:
correlation_matrix(df,num_cols)

# Çok güçlü korelasyonlar (0.8 ile 1 veya -0.8 ile -1.0 arası değerler)
# Güçlü korelasyonlar (0.6 ile 0.8 veya -0.6 ile -0.8 arası değerler)
# Orta derece korelasyonlar (0.4 ile 0.6 veya -0.4 ile -0.6 arası değerler)

IMDB Puanı en yüksek olan ilk 10 'Genre' hangileridir? Görselleştiriniz.

In [236]:
# formül
def top10genre_imdb(dataframe):
    dataframe= dataframe.groupby("Genre").agg({"IMDB_Score": "max"}).sort_values(by="IMDB_Score", ascending=False)
    return(dataframe.head(10))

# görselleştirmede kullanabilmek için:
topgenre = top10genre_imdb(df)
topgenre.reset_index(inplace=True)

# ilk 10 genreyi görelim:
print(topgenre)

In [237]:
f=plt.figure(figsize=(28,7))

plt.bar(topgenre.Genre,topgenre.IMDB_Score,width=0.5,
        color="#f05131",label="IMDB_Score")

plt.ylabel("IMDB_Score")
plt.xlabel("Genre")
plt.legend()
plt.title("IMDB Puanı en yüksek olan ilk 10 'Genre'")
plt.show()

'Runtime' değeri en yüksek olan ilk 10 film hangileridir? Görselleştiriniz.

In [238]:
# formül:
def runtime_title(dataframe):
    dataframe= dataframe.groupby("Title").agg({"Runtime": "max"}).sort_values(by="Runtime", ascending=False)
    runtime = dataframe.head(10)
    return(runtime)

# görselleştirmede kullanabilmek için:
runtimett = runtime_title(df)
runtimett.reset_index(inplace=True)

# ilk 10 filmi görelim:
print(runtimett)

In [239]:
f=plt.figure(figsize=(28,7))

plt.bar(runtimett.Title,runtimett.Runtime,width=0.5,
        color="#f05131",label="Runtime")

plt.ylabel("Runtime")
plt.xlabel("Title")
plt.legend()
plt.title("'Runtime' değeri en yüksek olan ilk 10 film")
plt.show()

Hangi yılda en fazla film yayımlanmıştır? Görselleştiriniz.

In [240]:
df['Year'] = df['Premiere'].dt.year
year = df['Year'].value_counts(sort=True)
year = year.to_frame().reset_index()
year.columns = ['Year', 'Count']

In [241]:
print(year)

In [242]:
f=plt.figure(figsize=(15,10))

plt.bar(year.Year,year.Count,width=0.5,
        color="#f05131",label="Count")

plt.ylabel("Count")
plt.xlabel("Year")
plt.legend()
plt.title("Yıllara göre yayımlanan film sayısı")
plt.show()

Hangi dilde yayımlanan filmler en düşük ortalama IMBD puanına sahiptir? Görselleştiriniz.

In [243]:
# formül:
def lowest_imdb(dataframe):
    dataframe= dataframe.groupby("Language").agg({"IMDB_Score": "mean"}).sort_values(by="IMDB_Score", ascending=True)
    runtime = dataframe.head(10)
    return(runtime)

# görselleştirmede kullanabilmek için:
low_imb = lowest_imdb(df)
low_imb.reset_index(inplace=True)

# en düşük imdbli dilleri görelim:
print(low_imb)

In [244]:

f=plt.figure(figsize=(12,5))

plt.barh(low_imb.Language,low_imb.IMDB_Score,
        color="#f05131",label="IMDB_Score")

plt.ylabel("Language")
plt.xlabel("IMDB_Score")
plt.legend()
plt.title("en düşük ortalama IMBD puanına sahip diller")
plt.show()

Hangi yılın toplam "runtime" süresi en fazladır?

In [245]:
df.groupby("Year").agg({"Runtime": "sum"}).sort_values(by="Runtime", ascending=False).head(1)

Her bir dilin en fazla kullanıldığı "Genre" nedir?

In [246]:
df.groupby(["Language","Genre"]).agg({"Language": "count"})

Veri setinde outlier veri var mıdır? Açıklayınız.

In [247]:
# aykırı gözlem analizi
# aykırı değer üst ve alt limitlerini bulma
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name, q1=0.25, q3=0.75):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1, q3)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [248]:
# aykırı gözlem varsa true
for col in num_cols:
    print(col, check_outlier(df, col))

In [249]:
# Grafik Teknikle Aykırı Değerler

sns.boxplot(x=df["IMDB_Score"])
plt.show()

sns.boxplot(x=df["Runtime"])
plt.show()

In [250]:
# aykırı değerleri üst ve alt limitler ile değiştirme
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [251]:
for col in num_cols:
    print(col, replace_with_thresholds(df, col))

In [252]:
# aykırı gözlemler düzeldi mi (false oldu mu)
for col in num_cols:
    print(col, check_outlier(df, col))

In [253]:
# Grafik Teknikle Baskıladığımız Aykırı Değerlere Tekrar bakalım

sns.boxplot(x=df["IMDB_Score"])
plt.show()

sns.boxplot(x=df["Runtime"])
plt.show()